In [8]:
%LOAD postgresql host=localhost dbname=farmacia user=farmacia password='caracteres' port=5432

In [9]:
CREATE TABLE comentario(
  tabla name NOT NULL,
  objeto_id integer NOT NULL,
  momento timestamp NOT NULL DEFAULT NOW(),
  comentario text NOT NULL,
  CONSTRAINT comentarios1_pkey PRIMARY KEY(
    tabla,
    objeto_id
  )
);

  CREATE INDEX comentario_momento_idx ON comentario(momento DESC);

Error: Cannot execute query. Fatal error. ERROR:  relation "comentario" already exists
 while executing "CREATE TABLE comentario(
  tabla name NOT NULL,
  objeto_id integer NOT NULL,
  momento timestamp NOT NULL DEFAULT NOW(),
  comentario text NOT NULL,
  CONSTRAINT comentarios1_pkey PRIMARY KEY(
    tabla,
    objeto_id
  )
);

  CREATE INDEX comentario_momento_idx ON comentario(momento DESC);".

In [10]:
ALTER TABLE comentario 
ADD CONSTRAINT zipchk CHECK ( EXISTS(SELECT name FROM information_schema.tables WHERE name = tabla)) NO INHERIT;

Error: Cannot execute query. Fatal error. ERROR:  cannot use subquery in check constraint
 while executing "ALTER TABLE comentario 
ADD CONSTRAINT zipchk CHECK ( EXISTS(SELECT name FROM information_schema.tables WHERE name = tabla)) NO INHERIT;".